This example uses CIFAR-10 data set. It can be downloaded at:
http://www.cs.toronto.edu/~kriz/cifar.html

Alex Krizhevsky wrote a tech report
http://www.cs.toronto.edu/~kriz/learning-features-2009-TR.pdf

In [1]:
from __future__ import print_function
import os
base_path = '/home/hadoop/data/cifar-10/cifar-10-batches-py'
train_files = ['data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5']
test_files = ['test_batch']

In [3]:
import tensorflow as tf
import numpy

def unpickle(file):
    import cPickle
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict

def dense_to_one_hot(labels_dense, num_classes):
      """Convert class labels from scalars to one-hot vectors."""
      num_labels = labels_dense.shape[0]
      index_offset = numpy.arange(num_labels) * num_classes
      labels_one_hot = numpy.zeros((num_labels, num_classes))
      labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
      return labels_one_hot

def load_data(files):
    # batches{'data', 'labels', 'batch_label', 'filenames'}
    batches = [unpickle(os.path.join(base_path, f)) for f in files]
    data = numpy.vstack((d['data'] for d in batches))
    data = numpy.reshape(data, [-1, 3, 32, 32]) / 255.0
    data = numpy.transpose(data, [0,2,3,1])
    label = numpy.vstack([numpy.expand_dims(numpy.asarray(d['labels']), axis=1) for d in batches])
    return data, dense_to_one_hot(label, 10)

train_data, train_label = load_data(train_files)
assert(train_data.shape == (50000, 32, 32, 3))
assert(train_label.shape == (50000, 10))

test_data, test_label = load_data(test_files)
assert(test_data.shape == (10000, 32, 32, 3))
assert(test_label.shape == (10000, 10))

def generate_batch(batch_size):
    perm = numpy.random.permutation(numpy.arange(len(train_data)))
    _train = train_data[perm]
    _label = train_label[perm]
    k = 1
    while k*batch_size <= len(train_data):
        start = (k-1) * batch_size
        end = k * batch_size
        yield _train[start:end], _label[start:end]
        k += 1

In [6]:
def cnn_model(is_training = True):
    # helper function for full connect layer
    def fc(name_scope, input_size, output_size, x):
        with tf.variable_scope(name_scope):
            w = tf.get_variable('w', [input_size, output_size], 
                                initializer=tf.random_uniform_initializer(minval=-1.0, maxval=1.0))
            b = tf.get_variable('b', [output_size], 
                                initializer=tf.constant_initializer(1))
            return tf.matmul(x,w) + b
    
    def conv(name_scope, filter_size, x):
        with tf.variable_scope(name_scope):
            w = tf.get_variable('filter', filter_size, 
                                initializer=tf.truncated_normal_initializer(stddev=0.1))
            c = tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding="VALID")
            
            b = tf.get_variable('b', [filter_size[-1]], 
                               initializer=tf.constant_initializer(1.0))
            return tf.nn.relu(tf.nn.bias_add(c, b))
            
    
    x = tf.placeholder(tf.float32, [None, 32, 32, 3])
    y = tf.placeholder(tf.float32, [None, 10])
    
    conv1 = conv('conv1', [5, 5, 3, 32], x)
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    norm1 = tf.nn.local_response_normalization(pool1, 4, bias=1.0, 
                                               alpha=0.001 / 9.0, beta=0.75, name='norm1')
    
    conv2 = conv('conv2', [5, 5, 32, 64], norm1)
    pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    norm2 = tf.nn.local_response_normalization(pool2, 4, bias=1.0, 
                                               alpha=0.001 / 9.0, beta=0.75, name='norm2')
    
#     pool_shape = pool2.get_shape().as_list()
#     shape = tf.Variable( [pool_shape[0], pool_shape[1]*pool_shape[2]*pool_shape[3]] )

    reshaped_pool = tf.reshape(norm2, shape=[-1, 5 * 5 * 64])
    
    fc1 = tf.nn.sigmoid(fc('fc1', 5 * 5 * 64, 256, reshaped_pool))
    if is_training:
        fc1 = tf.nn.dropout(fc1, keep_prob=0.5)
    
    fc2 = fc('fc2', 256, 10, fc1)
    logits = tf.nn.softmax(fc2)
    
    loss = -tf.reduce_mean(y * tf.log(logits))
    
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(logits,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # cifar-10 has only 10 classes
    # top_5 error below is only a reference; it's useful for ImageNet dataset
    top_5 = tf.nn.in_top_k(logits, tf.argmax(y,1), 5)
    accuracy_top_5 = tf.reduce_mean(tf.cast(top_5, tf.float32))
    
    if not is_training:
        return x, y, loss, accuracy, accuracy_top_5
    
    lr = 0.01
    train_step = tf.train.MomentumOptimizer(lr, 0.9).minimize(loss)
    return x, y, logits, loss, train_step

In [7]:
g = tf.Graph()
with g.as_default(), tf.Session() as session:
    with tf.variable_scope("model", reuse=None):
        x_train, y_train, train_logits, train_loss, train_step = cnn_model()
    with tf.variable_scope("model", reuse=True):
        x_test, y_test, test_loss, top1, top5 = cnn_model(is_training=False)
    
    session.run(tf.initialize_all_variables())
    batch_size = 512
    
    for i in range(1, 1001):
        _train_loss = 0
        batch_count = 0
        for _d, _l in generate_batch(batch_size):
            _loss, _ = session.run([train_loss, train_step], feed_dict={x_train:_d, y_train:_l})
            _train_loss += _loss
            batch_count += 1
        
        if i % 100 == 0:
            _test_loss, _top1, _top5 = session.run([test_loss, top1, top5], 
                                    feed_dict={x_test:test_data, y_test:test_label})

            print("{} epoch --> train_loss: {:.6f}  test_loss: {:.6f}  top1: {:.3f}  top5: {:.3f}".format(
                i, _train_loss/batch_count, _test_loss, _top1, _top5) )

100 epoch --> train_loss: 0.178052  test_loss: 0.163174  top1: 0.416  top5: 0.903
200 epoch --> train_loss: 0.161765  test_loss: 0.145139  top1: 0.476  top5: 0.927
300 epoch --> train_loss: 0.149602  test_loss: 0.139257  top1: 0.501  top5: 0.929
400 epoch --> train_loss: 0.141485  test_loss: 0.127108  top1: 0.551  top5: 0.947
500 epoch --> train_loss: 0.136091  test_loss: 0.124225  top1: 0.564  top5: 0.949
600 epoch --> train_loss: 0.130656  test_loss: 0.118713  top1: 0.584  top5: 0.953
700 epoch --> train_loss: 0.127984  test_loss: 0.121745  top1: 0.576  top5: 0.943
800 epoch --> train_loss: 0.124251  test_loss: 0.114829  top1: 0.589  top5: 0.959
900 epoch --> train_loss: 0.121454  test_loss: 0.112562  top1: 0.600  top5: 0.960
1000 epoch --> train_loss: 0.119200  test_loss: 0.110812  top1: 0.606  top5: 0.961
